In [1]:
import os
exec(open(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py')).read())

21/10/20 22:33:59 WARN Utils: Your hostname, Sumans-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.102 instead (on interface en0)
21/10/20 22:33:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/10/20 22:34:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.0.3
      /_/

Using Python version 3.8.2 (default, Apr  8 2021 23:19:18)
SparkSession available as 'spark'.


#### First let's read in the processed and cleansed data

In [2]:
sc.addFile('/Users/sumangangopadhyay/complex-spark-transformations/config.py')

In [3]:
import config as cf
from pyspark.sql import functions as func
from pyspark.sql.types import IntegerType

In [4]:
clean_data_path = cf.clean_data_path()
primary_response_variables = cf.primary_response_variables().split(',')
secondary_response_variables = cf.secondary_response_variables().split(',')
primary_explanatory_variables = cf.primary_explanatory_variables().split(',')

In [5]:
df = spark.read.csv(clean_data_path, header=True)

#### Let us look for seasonality, i.e. if the incidents of violation are more in particular month(s). Note how the Issue_Month column is sorted using type casting, since this column wasn't derived as an integer before

In [6]:
df.groupBy(df.Issue_Month).count().orderBy(df.Issue_Month.cast(IntegerType())).show()

21/10/20 22:34:16 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-------+
|Issue_Month|  count|
+-----------+-------+
|          1| 877365|
|          2| 826967|
|          3| 964737|
|          4| 888402|
|          5|1020244|
|          6| 852187|
|          7|    370|
|          8|    309|
|          9|    367|
|         10|    274|
|         11|    338|
|         12|    358|
+-----------+-------+



#### It is quite clear that the violations are most common during the first half of the year, so let's dig deeper in the first half

In [7]:
df_first6months = df.filter((df.Issue_Month.cast(IntegerType()) >= 1) \
                           | (df.Issue_Month.cast(IntegerType()) <= 6))

In [8]:
df_temp = df_first6months.groupBy(df_first6months.Issue_DayofMonth).count()

In [9]:
df_temp.columns

['Issue_DayofMonth', 'count']

In [10]:
df_temp.createOrReplaceTempView('t_count_per_day_of_month')

In [11]:
df_temp_standardised = spark.sql("select t1.Issue_DayofMonth, t1.count, \
round((t1.count - t2.avg_issue_count)/t2.stddev_issue_count,2) as standardised_issue_count \
from t_count_per_day_of_month t1 \
join (select avg(count) as avg_issue_count, stddev(count) as stddev_issue_count \
from t_count_per_day_of_month)t2 on 1=1")

In [13]:
df_temp_standardised.orderBy(df_temp_standardised.Issue_DayofMonth.cast(IntegerType())).show(40)

+----------------+------+------------------------+
|Issue_DayofMonth| count|standardised_issue_count|
+----------------+------+------------------------+
|               1|192311|                    0.53|
|               2|195915|                    0.64|
|               3|225398|                    1.56|
|               4|179977|                    0.15|
|               5|162463|                    -0.4|
|               6|211419|                    1.13|
|               7|179266|                    0.13|
|               8|187606|                    0.39|
|               9|170336|                   -0.15|
|              10|182476|                    0.23|
|              11|166438|                   -0.27|
|              12|162547|                    -0.4|
|              13|202336|                    0.84|
|              14|140262|                   -1.09|
|              15|171100|                   -0.13|
|              16|190189|                    0.47|
|              17|195997|      